## Package Import

In [ ]:
import pandas as pd
import openpyxl

## Set Pandas Display Options

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

## Data Inputs


In [ ]:
codetable = r"\\level\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\bctw-migrate\templates\inputs\code_table_20211021.csv"
codeheader = r"\\level\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\bctw-migrate\templates\inputs\codeheader_table_20211021.csv" 
codelookup = r"\\level\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\bctw-migrate\templates\inputs\codefieldlookup.csv"

In [ ]:
dfcodetable = pd.read_csv(codetable,index_col="code_header_id")
dfcodetable = dfcodetable[dfcodetable["valid_to"].isnull()]

dfcodeheader = pd.read_csv(codeheader,index_col="code_header_id")
dfcodeheader = dfcodeheader[dfcodeheader["valid_to"].isnull()]

dfcodelookup = pd.read_csv(codelookup,index_col='codeheader')

## Build Active Code Table
This table is built automatically by filtering out unused codes and code headers and then joining on the code map for which Casylys data goes to which db field.

In [ ]:
activecode = dfcodetable.join(dfcodeheader,on="code_header_id", rsuffix="_header",sort=True)
activecode = activecode[["code_id","code_name","code_description","code_header_name"]]

In [ ]:
activecode

In [ ]:
finalcodetable = activecode.join(dfcodelookup)
finalcodetable = finalcodetable[finalcodetable["dbfield"].notnull()]

In [ ]:
finalcodetable

In [ ]:
dfcodelookup= dfcodelookup[dfcodelookup.index.notnull()]

In [ ]:
dfcodelookup.head()

## Load Metadata sheet

In [ ]:
metadata= r"\\level\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\test_data\metadata_test_itch.xlsx"
metadatasheet = "Sheet1"

In [ ]:
fieldlist=['Region',
 'Species',
 'Caribou Population Unit',
 'WLH ID',
 'Animal ID',
 'Sex',
 'Life Stage',
 'Calf at Heel',
 'Ear Tag Right',
 'Ear Tag Left',
 'Device ID',
 'Radio Frequency',
 'Re-capture',
 'Reg_Key',
 'Trans-location',
 'Collar Type',
 'Collar Make',
 'Collar Model',
 'Satellite Network',
 'Capture Date',
 'Capture Date Year',
 'Capture Date Month',
 'Capture UTM Zone',
 'Capture UTM Easting',
 'Capture UTM Northing',
 'Capture Latitude',
 'Capture Longitude',
 'Release Date',
 'Animal Status',
 'Deployed',
 'Collar Status Details',
 'Deactivated?',
 'Collar Retrieved?',
 'Mortality Date',
 'Malfunction Date',
 'Malfunction Type',
 'Retrieval Date',
 'Mortality UTM Zone',
 'Mortality UTM Easting',
 'Mortality UTM Northing',
 'Mortality Latitude',
 'Mortality Longitude',
 'Max Transmission Date',
 'Previous Regional Comments',
 'Mortality Review',
 'Comments',
 'CID']

In [ ]:
dfmetadata = pd.read_excel(metadata,sheet_name=metadatasheet, engine='openpyxl',usecols=fieldlist)

In [ ]:
dfmetadata.head()

## Standardize Casylys form into BCTW values

In [ ]:
standardfields =['Animal Status', 'Calf at Heel', 'Caribou Population Unit', 'Collar Make', 'Deployed', 'Collar Status Details', 'Collar Type', 'Life Stage', 'Malfunction Type','Region', 'Satellite Network', 'Sex', 'Species']

## Load Casylys cleaning table

In [ ]:
casylyscodes = r"\\level\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\bctw-migrate\templates\inputs\casylyscodes.csv"

In [ ]:
dfcasylyscodes = pd.read_csv(casylyscodes,usecols=("DataField","DataFieldDescription","CodeforinDataField","BCTWCode","Coded","CodeMeaning"),index_col='DataField')
dfcasylyscodes.fillna(' ',inplace=True)

## Clean Casylys to BCTW Text

In [ ]:
for i in standardfields:
    x= dfmetadata[i]
    print(x)
    shortcodelist = dfcasylyscodes.loc[i]
    mapdict = dict(zip(shortcodelist.CodeforinDataField, shortcodelist.BCTWCode))
    print(mapdict)
    dfmetadata[i] = dfmetadata[i].map(mapdict)
    print(dfmetadata[i])
    

## Cell below uses the the active codes to reclassify metadata tables, output is in the same format as input but with BCTW codes for applicable fields

In [ ]:
for i in dfcodelookup.index.values:
    x = dfcodelookup['Casylys'][int(i)]
    print(("{} has been updated!").format(x))
    shortcodelist = finalcodetable.loc[int(i)]
    mapdict= dict(zip(shortcodelist.code_description, shortcodelist.code_id))
    dfmetadata[x] = dfmetadata[x].map(mapdict)

    


In [ ]:
dfmetadata

## Load BCTW output templates and prepare to have data migrated to them

In [ ]:
animal = r"\\level\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\bctw-migrate\templates\outputs\BCTW_bulk_import_animal_template.csv"
device = r"\\level\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\bctw-migrate\templates\outputs\BCTW_bulk_import_device_template.csv"

In [ ]:
dfanimal = pd.read_csv(animal)
dfdevice = pd.read_csv(device)
dffieldmap = pd.read_csv(codelookup)